In [1]:
globals().clear()

In [ ]:
import sys
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.discrete.discrete_model import Logit

In [7]:
df = pd.read_csv("/Users/terrylu/Desktop/UF/Courses/2025-2026/IO/IO_Python/data/coffee_data.csv")

### show data for the first 5 lines

In [11]:
for fname in [
    "/Users/terrylu/Desktop/UF/Courses/2025-2026/IO/IO_Python/data/coffee_data.csv"
]:
    print(f"\n>>> {fname} First 3 Lines:")
    print(pd.read_csv(fname, nrows=5))


>>> /Users/terrylu/Desktop/UF/Courses/2025-2026/IO/IO_Python/data/coffee_data.csv First 3 Lines:
   price_starbucks  price_dunkin  time_starbucks  time_dunkin     income  \
0         5.195254      4.993072              10           20  60.313563   
1         5.860757      2.720811               5           19  84.390030   
2         5.411054      3.556093               8           14  65.918877   
3         5.179533      2.150401               5            6  54.251162   
4         4.694619      2.047151               5           11  71.886693   

   choice_is_starbucks  
0                    1  
1                    0  
2                    1  
3                    0  
4                    0  


In [9]:
print(df.describe().T[['mean', 'std', 'min', 'max']])

                          mean        std       min         max
price_starbucks       4.985836   1.158422  3.000290    6.999912
price_dunkin          3.980958   1.165219  2.000666    5.999848
time_starbucks       12.511100   4.594604  5.000000   20.000000
time_dunkin          12.434200   4.590226  5.000000   20.000000
income               59.828028  15.027648 -9.899295  114.084881
choice_is_starbucks   0.324900   0.468361  0.000000    1.000000


Income is in 1000s

Travel time is in minutes

price is in dollars

Individual level data

Income varianle will be interacted with price

In [14]:
summary_by_choice = df.groupby('choice_is_starbucks').agg({
    'price_starbucks': 'mean',
    'price_dunkin': 'mean',
    'time_starbucks': 'mean',
    'time_dunkin': 'mean',
    'income': 'mean'
    }).rename(index={0: 'Chose Dunkin', 1: 'Chose Starbucks'})

print(summary_by_choice)

                     price_starbucks  price_dunkin  time_starbucks  \
choice_is_starbucks                                                  
Chose Dunkin                5.331770      3.634030       12.976152   
Chose Starbucks             4.267028      4.701831       11.544783   

                     time_dunkin     income  
choice_is_starbucks                          
Chose Dunkin           11.988891  59.666324  
Chose Starbucks        13.359495  60.164027  


create new variable for differences between product characteristics

In [15]:
df['price_diff'] = df['price_starbucks'] - df['price_dunkin']
df['time_diff'] = df['time_starbucks'] - df['time_dunkin']
df['price_income_interaction'] = df['price_starbucks'] * df['income']- df['price_dunkin'] * df['income']

y = df['choice_is_starbucks']
X = df[['price_diff', 'time_diff', 'price_income_interaction']]

note: no inercept here

In [18]:
logit_model = Logit(y, X)

logit_result = logit_model.fit(disp=True) # show iteration

print(logit_result.summary())

Optimization terminated successfully.
         Current function value: 0.363798
         Iterations 7
                            Logit Regression Results                           
Dep. Variable:     choice_is_starbucks   No. Observations:                10000
Model:                           Logit   Df Residuals:                     9997
Method:                            MLE   Df Model:                            2
Date:                 Wed, 10 Sep 2025   Pseudo R-squ.:                  0.4230
Time:                         14:30:35   Log-Likelihood:                -3638.0
converged:                        True   LL-Null:                       -6305.1
Covariance Type:             nonrobust   LLR p-value:                     0.000
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
price_diff                  -1.5569      0.109    -14.332      0.000    

price_income_interaction     0.0024: 

higer income people are less price sensitive

why: they care less about price

math why: the coefficient is positive, so as income increases, the negative effect of price_diff on the utility of choosing Starbucks is mitigated.

### With Robust Covariance Matrix

In [19]:
logit_result_robust = logit_model.fit(disp=True, cov_type='HC0')  # HC0 for robust standard errors
print(logit_result_robust.summary())

Optimization terminated successfully.
         Current function value: 0.363798
         Iterations 7
                            Logit Regression Results                           
Dep. Variable:     choice_is_starbucks   No. Observations:                10000
Model:                           Logit   Df Residuals:                     9997
Method:                            MLE   Df Model:                            2
Date:                 Wed, 10 Sep 2025   Pseudo R-squ.:                  0.4230
Time:                         14:36:08   Log-Likelihood:                -3638.0
converged:                        True   LL-Null:                       -6305.1
Covariance Type:                   HC0   LLR p-value:                     0.000
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
price_diff                  -1.5569      0.112    -13.955      0.000    

### Predicted probabilities

In [20]:
df['presdicted_prob_starbucks'] = logit_result.predict(X)
print(df[['choice_is_starbucks', 'presdicted_prob_starbucks']].head(10))

   choice_is_starbucks  presdicted_prob_starbucks
0                    1                   0.721069
1                    0                   0.073876
2                    1                   0.135187
3                    0                   0.014750
4                    0                   0.050792
5                    0                   0.718698
6                    0                   0.185674
7                    0                   0.002713
8                    0                   0.001230
9                    1                   0.594925


### Classification Rate
what's classification rate?: the percentage of correct predictions made by the model.

In [25]:
df['predicted_choice'] = (df['presdicted_prob_starbucks'] >= 0.5).astype(int)
# Calculate classification rate: the percentage of correct predictions
misclassified = (df['predicted_choice'] != df['choice_is_starbucks']).mean

print(misclassified)

<bound method Series.mean of 0       False
1       False
2        True
3       False
4       False
        ...  
9995     True
9996     True
9997    False
9998     True
9999    False
Length: 10000, dtype: bool>


### Marginal Effects

In [26]:
mfx = logit_result.get_margeff()
print(mfx.summary())

        Logit Marginal Effects        
Dep. Variable:     choice_is_starbucks
Method:                           dydx
At:                            overall
                              dy/dx    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
price_diff                  -0.1799      0.012    -14.888      0.000      -0.204      -0.156
time_diff                   -0.0143      0.001    -28.522      0.000      -0.015      -0.013
price_income_interaction     0.0003      0.000      1.372      0.170      -0.000       0.001


### Market Share Simulation (Price)

In [ ]:
df_sim = df.copy() # create a copy of the original dataframe for simulation
df_sim['price_starbucks_new'] = df_sim['price_starbucks'] + 1
df_sim['price_diff_new'] = df_sim['price_starbucks_new'] - df_sim['price_dunkin']

df_sim['price_income_interaction_new'] = df_sim['price_starbucks_new'] * df_sim['income'] - df_sim['price_dunkin'] * df_sim['income']

X_sim = df_sim[['price_diff_new', 'time_diff', 'price_income_interaction_new']] 
df_sim['predicted_prob_starbucks_new'] = logit_result.predict(X_sim)

old_market_share = df['choice_is_starbucks'].mean()
new_market_share = df_sim['predicted_prob_starbucks_new'].mean()
print(f"Old Market Share of Starbucks: {old_market_share:.4f}")
print(f"New Market Share of Starbucks after $1 price increase: {new_market_share:.4f}")

Old Market Share of Starbucks: 0.3249
New Market Share of Starbucks after $1 price increase: 0.1781


### Market Share Simulation (Income + 10% increase)

In [32]:
# income increase
df_sim2 = df.copy()
df_sim2['income_new'] = df_sim2['income'] * 1.1
df_sim2['price_income_interaction_new'] = df_sim2['price_starbucks'] * df_sim2['income_new'] - df_sim2['price_dunkin'] * df_sim2['income_new']

# new X variables
X_sim2 = df_sim2[['price_diff', 'time_diff', 'price_income_interaction_new']]

# predicted probabilities with new income
df_sim2['predicted_prob_starbucks_new2'] = logit_result.predict(X_sim2)

# comare market shares
old_market_share2 = df['choice_is_starbucks'].mean()
new_market_share2 = df_sim2['predicted_prob_starbucks_new2'].mean()
print(f"Old Market Share of Starbucks: {old_market_share2:.4f}")
print(f"New Market Share of Starbucks after 10% income increase: {new_market_share2:.4f}")  

Old Market Share of Starbucks: 0.3249
New Market Share of Starbucks after 10% income increase: 0.3221


## Try Logit By Hand

In [33]:
from scipy.optimize import minimize

The @ operator 

In [36]:
# Define Two Matrices
A = np.array([[1, 2], [3, 4]])
B = np.array([[5, 6], [7, 8]])

print(A @ B)  # Matrix multiplication

[[19 22]
 [43 50]]


Define our log-likelihood function

In [37]:
def log_likelihood(beta):
    """compute the log-likelihood for a binary logit model"""

    #compute mean utility for each observation
    V = X @ beta

    # logit choice Probabilities
    P = np.exp(V) / (1 + np.exp(V))

    # avoid log(0)
    esp = 1e-12
    p = np.clip(P, esp, 1 - esp)

    # log-likelihood
    logL = y * np.log(p) + (1 - y) * np.log(1 - p)

    return -np.sum(logL)  # Negative for minimization   

In [38]:
# Initialize parameters
beta_init = np.zeros(X.shape[1])

# optimize the likelihood
result = minimize(log_likelihood, beta_init, method='BFGS')

beta = result.x

print(beta)

[-1.55688815 -0.12355455  0.00237812]


## Try Probit By Hand

In [39]:
from scipy.stats import norm  # Import norm for Probit model

Define new likelihood function

In [40]:
def log_likelihood(beta):
    """compute the log-likelihood for a binary probit model"""

    #compute mean utility for each observation
    V = X @ beta

    # Probit choice Probabilities
    P = norm.cdf(V)

    # avoid log(0)
    esp = 1e-12
    p = np.clip(P, esp, 1 - esp)

    # log-likelihood
    logL = y * np.log(p) + (1 - y) * np.log(1 - p)

    return -np.sum(logL)  # Negative for minimization   

In [41]:
# Initialize parameters
beta_init = np.zeros(X.shape[1])

# optimize the likelihood
result = minimize(log_likelihood, beta_init, method='BFGS')

beta = result.x

print(beta)

[-0.863595   -0.06963015  0.00106199]
